<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
================================================================================
FASE 2D: ANÁLISIS DE CONFIGURACIONES
================================================================================
Trabajo Fin de Máster - Evaluación Comparativa de Técnicas de Segmentación
en Fotografía de Retrato

Autor: Jesús L. (Iesus)
Universidad: Universitat Oberta de Catalunya (UOC)
Máster: Data Science
Fecha: Diciembre 2025

Descripción:
    Este módulo implementa el análisis exhaustivo de configuraciones de los
    5 modelos de segmentación evaluados. Incluye:
    - Ranking global y por modelo
    - Análisis factorial por modelo (ANOVA, efectos principales e interacciones)
    - Análisis de sensibilidad a parámetros
    - Comparación de paradigmas arquitectónicos
    - Tests post-hoc (Tukey HSD)

Entrada:
    - metricas_fusionadas.csv (Fase 2B)
    - Archivos auxiliares de fases anteriores

Salida:
    - CSVs con análisis factorial por modelo
    - Rankings globales y por modelo
    - Análisis de sensibilidad
    - Comparación de paradigmas
    - JSON con resumen de hallazgos
================================================================================
"""

'\n================================================================================\nFASE 2D: ANÁLISIS DE CONFIGURACIONES\n================================================================================\nTrabajo Fin de Máster - Evaluación Comparativa de Técnicas de Segmentación\nen Fotografía de Retrato\n\nAutor: Jesús L. (Iesus)\nUniversidad: Universitat Oberta de Catalunya (UOC)\nMáster: Data Science\nFecha: Diciembre 2025\n\nDescripción:\n    Este módulo implementa el análisis exhaustivo de configuraciones de los\n    5 modelos de segmentación evaluados. Incluye:\n    - Ranking global y por modelo\n    - Análisis factorial por modelo (ANOVA, efectos principales e interacciones)\n    - Análisis de sensibilidad a parámetros\n    - Comparación de paradigmas arquitectónicos\n    - Tests post-hoc (Tukey HSD)\n\nEntrada:\n    - metricas_fusionadas.csv (Fase 2B)\n    - Archivos auxiliares de fases anteriores\n\nSalida:\n    - CSVs con análisis factorial por modelo\n    - Rankings globales

In [2]:
# =============================================================================
# IMPORTACIONES
# =============================================================================
import os
import sys
import json
import warnings
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field, asdict

import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import f_oneway, tukey_hsd, kruskal, mannwhitneyu
import logging

# Configuración de warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [3]:
# =============================================================================
# CONFIGURACIÓN DE LOGGING
# =============================================================================
def configurar_logging(nivel: int = logging.INFO) -> logging.Logger:
    """
    Configura el sistema de logging para el análisis.

    Parameters
    ----------
    nivel : int
        Nivel de logging (default: logging.INFO)

    Returns
    -------
    logging.Logger
        Logger configurado
    """
    # Eliminar handlers existentes para evitar duplicación en Colab
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    logging.basicConfig(
        level=nivel,
        format='[%(asctime)s] %(levelname)s - %(message)s',
        datefmt='%H:%M:%S'
    )

    logger = logging.getLogger(__name__)
    return logger

logger = configurar_logging()

In [4]:
# =============================================================================
# CONFIGURACIÓN DE RUTAS
# =============================================================================
@dataclass
class ConfiguracionRutas:
    """Configuración de rutas del proyecto."""

    # Detectar entorno (Colab o local)
    en_colab: bool = field(default_factory=lambda: 'google.colab' in sys.modules)

    def __post_init__(self):
        if self.en_colab:
            self.base = '/content/drive/MyDrive/TFM'
            self.datos_entrada = '/content/drive/MyDrive/TFM/3_Analisis/fase2b_correlaciones'
        else:
            # Rutas locales para desarrollo
            self.base = '/mnt/user-data/uploads'
            self.datos_entrada = '/mnt/user-data/uploads'

        self.salida = os.path.join(
            self.base if self.en_colab else '/home/claude',
            '3_Analisis' if self.en_colab else '',
            'fase2d_configuraciones'
        )

    def crear_directorios(self):
        """Crea los directorios de salida si no existen."""
        os.makedirs(self.salida, exist_ok=True)
        logger.info(f"Directorio de salida: {self.salida}")

In [5]:
# =============================================================================
# ESTRUCTURAS DE DATOS
# =============================================================================
@dataclass
class ResultadoANOVA:
    """Resultado de un análisis ANOVA."""
    factor: str
    f_statistic: float
    p_valor: float
    eta_squared: float
    omega_squared: float
    n_grupos: int
    n_total: int
    significativo: bool
    interpretacion_efecto: str

@dataclass
class ResultadoPostHoc:
    """Resultado de comparación post-hoc."""
    grupo_1: str
    grupo_2: str
    diferencia_medias: float
    p_valor: float
    significativo: bool
    ic_inferior: float
    ic_superior: float

@dataclass
class AnalisisFactorial:
    """Resultado completo de análisis factorial para un modelo."""
    modelo: str
    factores: List[str]
    anova_resultados: List[ResultadoANOVA]
    posthoc_resultados: List[ResultadoPostHoc]
    mejor_configuracion: str
    iou_mejor: float
    estadisticas_por_nivel: Dict[str, Dict]

In [6]:
# =============================================================================
# CLASE PRINCIPAL: ANALIZADOR DE CONFIGURACIONES
# =============================================================================
class AnalizadorConfiguraciones:
    """
    Clase principal para el análisis de configuraciones de modelos.

    Implementa análisis factorial, sensibilidad y comparación de paradigmas
    para los modelos de segmentación evaluados.

    Nota: SAM2 se analiza de forma unificada (sam2 + sam2_prompts) para
    permitir comparación entre modo automático y modo con prompts.
    """

    # Definición de factores por modelo
    FACTORES_MODELO = {
        'bodypix': {
            'multiplicador': lambda x: x.split('_')[3],  # 050, 075
            'nivel_sensibilidad': lambda x: '_'.join(x.split('_')[4:-1]).replace('_t0', ''),
            'umbral': lambda x: x.split('_')[-1]
        },
        'mask2former': {
            'backbone': lambda x: x.split('_')[1],  # base, large, tiny
            'dataset': lambda x: x.split('_')[2],  # ade, coco
            'sensibilidad': lambda x: '_'.join(x.split('_')[3:])
        },
        'oneformer': {
            'dataset': lambda x: x.split('_')[1],  # ade20k, coco
            'backbone': lambda x: x.split('_')[2],  # swin, tiny
            'task_type': lambda x: x.split('_')[3],  # instance, panoptic, semantic
            'umbral': lambda x: x.split('_')[4]  # t040, t060, etc
        },
        'sam2': {
            'modo': lambda x: 'prompts' if 'prompts' in x else 'auto',
            'tamano': lambda x: AnalizadorConfiguraciones._extraer_tamano_sam2(x),
            'estrategia': lambda x: AnalizadorConfiguraciones._extraer_estrategia_sam2(x)
        },
        'yolov8': {
            'tamano': lambda x: x.split('_')[1],  # nano, small, medium, large, xlarge
            'config_sensibilidad': lambda x: x.split('_')[2]  # balanced, fast, quality, sensitive
        }
    }

    # Paradigmas arquitectónicos
    PARADIGMAS = {
        'cnn_especializada': ['yolov8'],
        'transformer_segmentacion': ['mask2former', 'oneformer'],
        'foundation_model': ['sam2', 'sam2_prompts'],  # Incluir ambos
        'web_ligero': ['bodypix']
    }

    def __init__(self, df: pd.DataFrame, rutas: ConfiguracionRutas):
        """
        Inicializa el analizador.

        Parameters
        ----------
        df : pd.DataFrame
            DataFrame con métricas fusionadas
        rutas : ConfiguracionRutas
            Configuración de rutas
        """
        self.df = df.copy()
        self.rutas = rutas
        self.resultados = {}

        # Crear columna auxiliar para análisis unificado de SAM2
        self.df['modelo_analisis'] = self.df['modelo'].replace('sam2_prompts', 'sam2')

        # Extraer factores para cada modelo
        self._extraer_factores()

        logger.info(f"Analizador inicializado con {len(df)} evaluaciones")
        logger.info(f"Modelos originales: {df['modelo'].unique().tolist()}")
        logger.info(f"Modelos para análisis: {self.df['modelo_analisis'].unique().tolist()}")
        logger.info(f"Configuraciones únicas: {df['config_codigo'].nunique()}")

    @staticmethod
    def _extraer_tamano_sam2(config: str) -> str:
        """
        Extrae el tamaño del modelo SAM2 de la configuración.

        Patrones:
        - Auto: sam2_{tamaño}_{config} → sam2_base_plus_balanced
        - Prompts: sam2_prompts_{tamaño}_{estrategia} → sam2_prompts_base_plus_saliency_moderate
        """
        if 'base_plus' in config:
            return 'base_plus'
        elif 'tiny' in config:
            return 'tiny'
        elif 'small' in config:
            return 'small'
        elif 'large' in config:
            return 'large'
        return 'unknown'

    @staticmethod
    def _extraer_estrategia_sam2(config: str) -> str:
        """
        Extrae la estrategia de SAM2 de la configuración.

        Modo automático (sam2_{tamaño}_{config}):
        - balanced, low_cost, quality

        Modo con prompts (sam2_prompts_{tamaño}_{estrategia}):
        - bbox_heuristic, combined_aggressive, combined_moderate
        - grid_central_aggressive, grid_central_conservative, grid_central_moderate
        - saliency_conservative, saliency_moderate
        """
        if 'prompts' not in config:
            # Modo automático
            if 'low_cost' in config:
                return 'low_cost'
            elif 'balanced' in config:
                return 'balanced'
            elif 'quality' in config:
                return 'quality'
            return 'unknown'
        else:
            # Modo con prompts
            sin_prefijo = config.replace('sam2_prompts_', '')
            for tamano in ['base_plus_', 'tiny_', 'small_', 'large_']:
                if sin_prefijo.startswith(tamano):
                    return sin_prefijo.replace(tamano, '', 1)
            return sin_prefijo

    def _extraer_factores(self):
        """Extrae los factores de cada configuración según el modelo."""
        logger.info("Extrayendo factores de configuraciones...")

        for modelo, extractores in self.FACTORES_MODELO.items():
            # Usar modelo_analisis para incluir sam2_prompts en análisis de sam2
            mask = self.df['modelo_analisis'] == modelo
            if not mask.any():
                continue

            for factor, extractor in extractores.items():
                col_name = f'factor_{factor}'
                self.df.loc[mask, col_name] = self.df.loc[mask, 'config_codigo'].apply(
                    lambda x: self._safe_extract(extractor, x)
                )

        # Logging de factores extraídos
        for modelo in self.df['modelo_analisis'].unique():
            df_modelo = self.df[self.df['modelo_analisis'] == modelo]
            factores = [c for c in df_modelo.columns if c.startswith('factor_')]
            for f in factores:
                valores = df_modelo[f].dropna().unique()
                if len(valores) > 0:
                    logger.info(f"  {modelo}.{f}: {sorted(valores)}")

    @staticmethod
    def _safe_extract(extractor, valor):
        """Extrae un factor de forma segura."""
        try:
            return extractor(valor)
        except Exception:
            return None

    # =========================================================================
    # RANKING DE CONFIGURACIONES
    # =========================================================================
    def calcular_ranking_global(self, top_n: int = 30) -> pd.DataFrame:
        """
        Calcula el ranking global de configuraciones por IoU.

        Parameters
        ----------
        top_n : int
            Número de configuraciones top a incluir

        Returns
        -------
        pd.DataFrame
            Ranking de configuraciones
        """
        logger.info(f"Calculando ranking global TOP-{top_n}...")

        # Agrupar por configuración (usar modelo original, no modelo_analisis)
        ranking = self.df.groupby(['modelo', 'config_codigo']).agg({
            'iou': ['mean', 'std', 'min', 'max', 'count'],
            'dice': 'mean',
            'precision': 'mean',
            'recall': 'mean'
        }).reset_index()

        # Aplanar columnas
        ranking.columns = [
            'modelo', 'config_codigo',
            'iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n_fotos',
            'dice_mean', 'precision_mean', 'recall_mean'
        ]

        # Calcular coeficiente de variación
        ranking['iou_cv'] = ranking['iou_std'] / ranking['iou_mean']

        # Ordenar por IoU medio descendente
        ranking = ranking.sort_values('iou_mean', ascending=False)

        # Agregar posición en ranking
        ranking['posicion'] = range(1, len(ranking) + 1)

        # Guardar ranking completo
        self.resultados['ranking_global'] = ranking.copy()

        # Retornar top N
        return ranking.head(top_n)

    def calcular_ranking_por_modelo(self) -> pd.DataFrame:
        """
        Calcula la mejor configuración por modelo.

        Returns
        -------
        pd.DataFrame
            Mejor configuración de cada modelo
        """
        logger.info("Calculando mejor configuración por modelo...")

        mejores = []

        # Uso modelo original para mantener distinción sam2 vs sam2_prompts
        for modelo in self.df['modelo'].unique():
            df_modelo = self.df[self.df['modelo'] == modelo]

            stats_config = df_modelo.groupby('config_codigo').agg({
                'iou': ['mean', 'std', 'min', 'max', 'count']
            }).reset_index()
            stats_config.columns = ['config_codigo', 'iou_mean', 'iou_std',
                                    'iou_min', 'iou_max', 'n_fotos']

            mejor = stats_config.loc[stats_config['iou_mean'].idxmax()]

            mejores.append({
                'modelo': modelo,
                'mejor_config': mejor['config_codigo'],
                'iou_mean': mejor['iou_mean'],
                'iou_std': mejor['iou_std'],
                'iou_min': mejor['iou_min'],
                'iou_max': mejor['iou_max'],
                'n_fotos': int(mejor['n_fotos']),
                'n_configuraciones_total': df_modelo['config_codigo'].nunique()
            })

        resultado = pd.DataFrame(mejores)
        resultado = resultado.sort_values('iou_mean', ascending=False)

        self.resultados['ranking_por_modelo'] = resultado

        return resultado

    # =========================================================================
    # ANÁLISIS FACTORIAL POR MODELO
    # =========================================================================
    def _calcular_anova(self, df: pd.DataFrame, factor: str,
                        metrica: str = 'iou') -> ResultadoANOVA:
        """
        Calcula ANOVA de un factor.

        Parameters
        ----------
        df : pd.DataFrame
            Datos del modelo
        factor : str
            Nombre del factor (columna)
        metrica : str
            Métrica a analizar

        Returns
        -------
        ResultadoANOVA
            Resultado del análisis
        """
        col_factor = f'factor_{factor}' if not factor.startswith('factor_') else factor

        if col_factor not in df.columns:
            return None

        df_clean = df[[col_factor, metrica]].dropna()

        if df_clean[col_factor].nunique() < 2:
            return None

        grupos = [grupo[metrica].values for _, grupo in df_clean.groupby(col_factor)]

        if len(grupos) < 2:
            return None

        try:
            f_stat, p_valor = f_oneway(*grupos)
        except Exception:
            return None

        n_total = len(df_clean)
        n_grupos = len(grupos)

        grand_mean = df_clean[metrica].mean()
        ss_between = sum(len(g) * (g.mean() - grand_mean)**2 for g in grupos)
        ss_total = ((df_clean[metrica] - grand_mean)**2).sum()
        ss_within = ss_total - ss_between

        eta_sq = ss_between / ss_total if ss_total > 0 else 0

        df_between = n_grupos - 1
        ms_within = ss_within / (n_total - n_grupos) if (n_total - n_grupos) > 0 else 0
        omega_sq = (ss_between - df_between * ms_within) / (ss_total + ms_within)
        omega_sq = max(0, omega_sq)

        if eta_sq < 0.01:
            interpretacion = 'insignificante'
        elif eta_sq < 0.06:
            interpretacion = 'pequeno'
        elif eta_sq < 0.14:
            interpretacion = 'mediano'
        else:
            interpretacion = 'grande'

        return ResultadoANOVA(
            factor=factor,
            f_statistic=float(f_stat),
            p_valor=float(p_valor),
            eta_squared=float(eta_sq),
            omega_squared=float(omega_sq),
            n_grupos=n_grupos,
            n_total=n_total,
            significativo=p_valor < 0.05,
            interpretacion_efecto=interpretacion
        )

    def _calcular_posthoc_tukey(self, df: pd.DataFrame, factor: str,
                                 metrica: str = 'iou') -> List[ResultadoPostHoc]:
        """
        Calcula comparaciones post-hoc Tukey HSD.

        Parameters
        ----------
        df : pd.DataFrame
            Datos del modelo
        factor : str
            Nombre del factor
        metrica : str
            Métrica a analizar

        Returns
        -------
        List[ResultadoPostHoc]
            Lista de comparaciones pareadas
        """
        col_factor = f'factor_{factor}' if not factor.startswith('factor_') else factor

        if col_factor not in df.columns:
            return []

        df_clean = df[[col_factor, metrica]].dropna()

        if df_clean[col_factor].nunique() < 2:
            return []

        grupos = df_clean.groupby(col_factor)[metrica].apply(list).to_dict()
        nombres = list(grupos.keys())
        datos = [np.array(grupos[n]) for n in nombres]

        if len(datos) < 2:
            return []

        try:
            resultado_tukey = tukey_hsd(*datos)
        except Exception:
            return []

        resultados = []

        for i in range(len(nombres)):
            for j in range(i + 1, len(nombres)):
                diff = np.mean(datos[i]) - np.mean(datos[j])
                p_val = resultado_tukey.pvalue[i, j]
                ci = resultado_tukey.confidence_interval(confidence_level=0.95)
                ci_low = ci.low[i, j]
                ci_high = ci.high[i, j]

                resultados.append(ResultadoPostHoc(
                    grupo_1=str(nombres[i]),
                    grupo_2=str(nombres[j]),
                    diferencia_medias=float(diff),
                    p_valor=float(p_val),
                    significativo=p_val < 0.05,
                    ic_inferior=float(ci_low),
                    ic_superior=float(ci_high)
                ))

        return resultados

    def analizar_bodypix(self) -> Dict:
        """
        Análisis factorial completo para BodyPix.

        Factores:
        - multiplicador: 050, 075
        - nivel_sensibilidad: ultra_sensible, sensibilidad_alta, sensibilidad_media, baja_sensibilidad
        - umbral: valores numéricos

        Returns
        -------
        Dict
            Resultados del análisis
        """
        logger.info("Analizando BodyPix...")

        df_modelo = self.df[self.df['modelo'] == 'bodypix'].copy()

        resultados = {
            'modelo': 'bodypix',
            'n_evaluaciones': len(df_modelo),
            'n_configuraciones': df_modelo['config_codigo'].nunique(),
            'factores': {}
        }

        for factor in ['multiplicador', 'nivel_sensibilidad', 'umbral']:
            col = f'factor_{factor}'
            if col not in df_modelo.columns:
                continue

            anova = self._calcular_anova(df_modelo, factor)

            stats_nivel = df_modelo.groupby(col)['iou'].agg([
                'mean', 'std', 'min', 'max', 'count'
            ]).reset_index()
            stats_nivel.columns = [factor, 'iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n']

            posthoc = []
            if anova and anova.significativo:
                posthoc = self._calcular_posthoc_tukey(df_modelo, factor)

            resultados['factores'][factor] = {
                'anova': asdict(anova) if anova else None,
                'estadisticas_nivel': stats_nivel.to_dict('records'),
                'posthoc': [asdict(p) for p in posthoc]
            }

        if 'factor_multiplicador' in df_modelo.columns and 'factor_nivel_sensibilidad' in df_modelo.columns:
            interaccion = df_modelo.groupby(
                ['factor_multiplicador', 'factor_nivel_sensibilidad']
            )['iou'].agg(['mean', 'std', 'count']).reset_index()
            interaccion.columns = ['multiplicador', 'nivel_sensibilidad', 'iou_mean', 'iou_std', 'n']
            resultados['interaccion_multiplicador_sensibilidad'] = interaccion.to_dict('records')

        mejor = df_modelo.groupby('config_codigo')['iou'].mean().idxmax()
        resultados['mejor_configuracion'] = mejor
        resultados['iou_mejor'] = float(df_modelo.groupby('config_codigo')['iou'].mean().max())

        return resultados

    def analizar_mask2former(self) -> Dict:
        """
        Análisis factorial completo para Mask2Former.

        Factores:
        - backbone: base, large, tiny
        - dataset: ade, coco
        - sensibilidad: varios niveles

        Hallazgo crítico esperado: COCO produce IoU=0 en muchas configuraciones

        Returns
        -------
        Dict
            Resultados del análisis
        """
        logger.info("Analizando Mask2Former...")

        df_modelo = self.df[self.df['modelo'] == 'mask2former'].copy()

        resultados = {
            'modelo': 'mask2former',
            'n_evaluaciones': len(df_modelo),
            'n_configuraciones': df_modelo['config_codigo'].nunique(),
            'factores': {}
        }

        configs_cero = df_modelo.groupby('config_codigo')['iou'].mean()
        configs_cero = configs_cero[configs_cero == 0].index.tolist()
        resultados['configuraciones_iou_cero'] = configs_cero
        resultados['n_configs_iou_cero'] = len(configs_cero)

        for factor in ['backbone', 'dataset', 'sensibilidad']:
            col = f'factor_{factor}'
            if col not in df_modelo.columns:
                continue

            anova = self._calcular_anova(df_modelo, factor)

            stats_nivel = df_modelo.groupby(col)['iou'].agg([
                'mean', 'std', 'min', 'max', 'count'
            ]).reset_index()
            stats_nivel.columns = [factor, 'iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n']

            posthoc = []
            if anova and anova.significativo:
                posthoc = self._calcular_posthoc_tukey(df_modelo, factor)

            resultados['factores'][factor] = {
                'anova': asdict(anova) if anova else None,
                'estadisticas_nivel': stats_nivel.to_dict('records'),
                'posthoc': [asdict(p) for p in posthoc]
            }

        if 'factor_dataset' in df_modelo.columns:
            comparacion_dataset = df_modelo.groupby('factor_dataset').agg({
                'iou': ['mean', 'std', 'min', 'max', 'count'],
                'config_codigo': 'nunique'
            }).reset_index()
            comparacion_dataset.columns = [
                'dataset', 'iou_mean', 'iou_std', 'iou_min', 'iou_max',
                'n_evaluaciones', 'n_configuraciones'
            ]
            resultados['comparacion_ade_coco'] = comparacion_dataset.to_dict('records')

            ade_data = df_modelo[df_modelo['factor_dataset'] == 'ade']['iou']
            coco_data = df_modelo[df_modelo['factor_dataset'] == 'coco']['iou']

            if len(ade_data) > 0 and len(coco_data) > 0:
                t_stat, p_val = stats.ttest_ind(ade_data, coco_data)
                pooled_std = np.sqrt((ade_data.std()**2 + coco_data.std()**2) / 2)
                cohens_d = (ade_data.mean() - coco_data.mean()) / pooled_std if pooled_std > 0 else 0
                resultados['test_ade_vs_coco'] = {
                    't_statistic': float(t_stat),
                    'p_valor': float(p_val),
                    'cohens_d': float(cohens_d),
                    'ade_mean': float(ade_data.mean()),
                    'coco_mean': float(coco_data.mean()),
                    'diferencia': float(ade_data.mean() - coco_data.mean())
                }

        mejor = df_modelo.groupby('config_codigo')['iou'].mean().idxmax()
        resultados['mejor_configuracion'] = mejor
        resultados['iou_mejor'] = float(df_modelo.groupby('config_codigo')['iou'].mean().max())

        return resultados

    def analizar_oneformer(self) -> Dict:
        """
        Análisis factorial completo para OneFormer.

        Factores:
        - dataset: ade20k, coco
        - backbone: swin, tiny
        - task_type: instance, panoptic, semantic
        - umbral: t040, t060, t075, t085

        Returns
        -------
        Dict
            Resultados del análisis
        """
        logger.info("Analizando OneFormer...")

        df_modelo = self.df[self.df['modelo'] == 'oneformer'].copy()

        resultados = {
            'modelo': 'oneformer',
            'n_evaluaciones': len(df_modelo),
            'n_configuraciones': df_modelo['config_codigo'].nunique(),
            'factores': {}
        }

        for factor in ['dataset', 'backbone', 'task_type', 'umbral']:
            col = f'factor_{factor}'
            if col not in df_modelo.columns:
                continue

            anova = self._calcular_anova(df_modelo, factor)

            stats_nivel = df_modelo.groupby(col)['iou'].agg([
                'mean', 'std', 'min', 'max', 'count'
            ]).reset_index()
            stats_nivel.columns = [factor, 'iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n']

            posthoc = []
            if anova and anova.significativo:
                posthoc = self._calcular_posthoc_tukey(df_modelo, factor)

            resultados['factores'][factor] = {
                'anova': asdict(anova) if anova else None,
                'estadisticas_nivel': stats_nivel.to_dict('records'),
                'posthoc': [asdict(p) for p in posthoc]
            }

        if all(f'factor_{f}' in df_modelo.columns for f in ['dataset', 'backbone', 'task_type']):
            interaccion = df_modelo.groupby(
                ['factor_dataset', 'factor_backbone', 'factor_task_type']
            )['iou'].agg(['mean', 'std', 'count']).reset_index()
            interaccion.columns = ['dataset', 'backbone', 'task_type', 'iou_mean', 'iou_std', 'n']
            resultados['interaccion_completa'] = interaccion.to_dict('records')

        if 'factor_task_type' in df_modelo.columns and 'factor_umbral' in df_modelo.columns:
            sensibilidad_umbral = df_modelo.groupby(
                ['factor_task_type', 'factor_umbral']
            )['iou'].agg(['mean', 'std']).reset_index()
            sensibilidad_umbral.columns = ['task_type', 'umbral', 'iou_mean', 'iou_std']
            resultados['sensibilidad_umbral_por_task'] = sensibilidad_umbral.to_dict('records')

        mejor = df_modelo.groupby('config_codigo')['iou'].mean().idxmax()
        resultados['mejor_configuracion'] = mejor
        resultados['iou_mejor'] = float(df_modelo.groupby('config_codigo')['iou'].mean().max())

        return resultados

    def analizar_sam2(self) -> Dict:
        """
        Análisis factorial completo para SAM2.

        Incluye tanto sam2 (modo auto) como sam2_prompts (modo con prompts).

        Factores:
        - modo: auto, prompts
        - tamano: tiny, small, base_plus, large
        - estrategia: varias según el modo

        Returns
        -------
        Dict
            Resultados del análisis
        """
        logger.info("Analizando SAM2 (auto + prompts)...")

        # Incluir tanto sam2 como sam2_prompts
        df_modelo = self.df[self.df['modelo'].isin(['sam2', 'sam2_prompts'])].copy()

        resultados = {
            'modelo': 'sam2',
            'n_evaluaciones': len(df_modelo),
            'n_configuraciones': df_modelo['config_codigo'].nunique(),
            'n_evaluaciones_auto': len(df_modelo[df_modelo['modelo'] == 'sam2']),
            'n_evaluaciones_prompts': len(df_modelo[df_modelo['modelo'] == 'sam2_prompts']),
            'factores': {}
        }

        # Análisis por factor principal
        for factor in ['modo', 'tamano']:
            col = f'factor_{factor}'
            if col not in df_modelo.columns:
                continue

            anova = self._calcular_anova(df_modelo, factor)

            stats_nivel = df_modelo.groupby(col)['iou'].agg([
                'mean', 'std', 'min', 'max', 'count'
            ]).reset_index()
            stats_nivel.columns = [factor, 'iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n']

            posthoc = []
            if anova and anova.significativo:
                posthoc = self._calcular_posthoc_tukey(df_modelo, factor)

            resultados['factores'][factor] = {
                'anova': asdict(anova) if anova else None,
                'estadisticas_nivel': stats_nivel.to_dict('records'),
                'posthoc': [asdict(p) for p in posthoc]
            }

        # Análisis detallado: auto vs prompts
        if 'factor_modo' in df_modelo.columns:
            auto_data = df_modelo[df_modelo['factor_modo'] == 'auto']['iou']
            prompts_data = df_modelo[df_modelo['factor_modo'] == 'prompts']['iou']

            if len(auto_data) > 0 and len(prompts_data) > 0:
                t_stat, p_val = stats.ttest_ind(auto_data, prompts_data)
                pooled_std = np.sqrt((auto_data.std()**2 + prompts_data.std()**2) / 2)
                cohens_d = (prompts_data.mean() - auto_data.mean()) / pooled_std if pooled_std > 0 else 0
                resultados['comparacion_auto_prompts'] = {
                    't_statistic': float(t_stat),
                    'p_valor': float(p_val),
                    'cohens_d': float(cohens_d),
                    'auto_mean': float(auto_data.mean()),
                    'auto_std': float(auto_data.std()),
                    'prompts_mean': float(prompts_data.mean()),
                    'prompts_std': float(prompts_data.std()),
                    'diferencia': float(prompts_data.mean() - auto_data.mean()),
                    'mejora_porcentual': float((prompts_data.mean() - auto_data.mean()) / auto_data.mean() * 100) if auto_data.mean() > 0 else None
                }

        # Análisis de estrategias de prompts
        df_prompts = df_modelo[df_modelo['factor_modo'] == 'prompts'].copy()
        if len(df_prompts) > 0 and 'factor_estrategia' in df_prompts.columns:
            stats_estrategia = df_prompts.groupby('factor_estrategia')['iou'].agg([
                'mean', 'std', 'min', 'max', 'count'
            ]).reset_index()
            stats_estrategia.columns = ['estrategia', 'iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n']
            stats_estrategia = stats_estrategia.sort_values('iou_mean', ascending=False)
            resultados['estrategias_prompts'] = stats_estrategia.to_dict('records')

            anova_estrategia = self._calcular_anova(df_prompts, 'estrategia')
            if anova_estrategia:
                resultados['anova_estrategias'] = asdict(anova_estrategia)

            # Categorizar estrategias por tipo
            def categorizar_estrategia(est):
                if pd.isna(est):
                    return 'otra'
                if 'saliency' in str(est):
                    return 'saliencia'
                elif 'grid' in str(est):
                    return 'grid'
                elif 'combined' in str(est):
                    return 'combinada'
                elif 'bbox' in str(est):
                    return 'bbox'
                return 'otra'

            df_prompts['categoria_estrategia'] = df_prompts['factor_estrategia'].apply(categorizar_estrategia)

            stats_categoria = df_prompts.groupby('categoria_estrategia')['iou'].agg([
                'mean', 'std', 'min', 'max', 'count'
            ]).reset_index()
            stats_categoria.columns = ['categoria', 'iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n']
            stats_categoria = stats_categoria.sort_values('iou_mean', ascending=False)
            resultados['categorias_estrategia'] = stats_categoria.to_dict('records')

            # ANOVA por categoría
            if df_prompts['categoria_estrategia'].nunique() >= 2:
                anova_categoria = self._calcular_anova(df_prompts, 'categoria_estrategia')
                if anova_categoria:
                    resultados['anova_categorias'] = asdict(anova_categoria)

        # Análisis modo automático por configuración
        df_auto = df_modelo[df_modelo['factor_modo'] == 'auto'].copy()
        if len(df_auto) > 0 and 'factor_estrategia' in df_auto.columns:
            stats_auto = df_auto.groupby('factor_estrategia')['iou'].agg([
                'mean', 'std', 'count'
            ]).reset_index()
            stats_auto.columns = ['config_auto', 'iou_mean', 'iou_std', 'n']
            resultados['configuraciones_auto'] = stats_auto.to_dict('records')

        # Interacción modo × tamaño
        if 'factor_modo' in df_modelo.columns and 'factor_tamano' in df_modelo.columns:
            interaccion = df_modelo.groupby(
                ['factor_modo', 'factor_tamano']
            )['iou'].agg(['mean', 'std', 'count']).reset_index()
            interaccion.columns = ['modo', 'tamano', 'iou_mean', 'iou_std', 'n']
            resultados['interaccion_modo_tamano'] = interaccion.to_dict('records')

        # Análisis por tamaño SOLO en modo prompts
        if len(df_prompts) > 0 and 'factor_tamano' in df_prompts.columns:
            stats_tamano_prompts = df_prompts.groupby('factor_tamano')['iou'].agg([
                'mean', 'std', 'min', 'max', 'count'
            ]).reset_index()
            stats_tamano_prompts.columns = ['tamano', 'iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n']
            resultados['tamano_en_prompts'] = stats_tamano_prompts.to_dict('records')

            anova_tamano_prompts = self._calcular_anova(df_prompts, 'tamano')
            if anova_tamano_prompts:
                resultados['anova_tamano_prompts'] = asdict(anova_tamano_prompts)

        # Análisis cruzado: mejor estrategia por tamaño
        if len(df_prompts) > 0 and 'factor_tamano' in df_prompts.columns and 'factor_estrategia' in df_prompts.columns:
            mejor_por_tamano = []
            for tamano in df_prompts['factor_tamano'].dropna().unique():
                df_tam = df_prompts[df_prompts['factor_tamano'] == tamano]
                if len(df_tam) > 0:
                    stats_est = df_tam.groupby('factor_estrategia')['iou'].mean()
                    if len(stats_est) > 0:
                        mejor_est = stats_est.idxmax()
                        mejor_iou = stats_est.max()
                        mejor_por_tamano.append({
                            'tamano': tamano,
                            'mejor_estrategia': mejor_est,
                            'iou_mean': float(mejor_iou)
                        })
            resultados['mejor_estrategia_por_tamano'] = mejor_por_tamano

        # Mejor configuración global
        stats_config = df_modelo.groupby('config_codigo')['iou'].mean()
        mejor = stats_config.idxmax()
        resultados['mejor_configuracion'] = mejor
        resultados['iou_mejor'] = float(stats_config.max())

        # Hallazgo clave: saliencia como mejor enfoque
        if 'estrategias_prompts' in resultados and len(resultados['estrategias_prompts']) > 0:
            top_estrategia = resultados['estrategias_prompts'][0]
            resultados['hallazgo_clave'] = {
                'mejor_estrategia': top_estrategia['estrategia'],
                'iou': top_estrategia['iou_mean'],
                'es_saliencia': 'saliency' in str(top_estrategia['estrategia']),
                'interpretacion': 'Las estrategias basadas en saliencia visual superan significativamente a otras aproximaciones, validando el uso de conocimiento de dominio fotografico.'
            }

        return resultados

    def analizar_yolov8(self) -> Dict:
        """
        Análisis factorial completo para YOLOv8.

        Factores:
        - tamano: nano, small, medium, large, xlarge
        - config_sensibilidad: balanced, fast, quality, sensitive

        Returns
        -------
        Dict
            Resultados del análisis
        """
        logger.info("Analizando YOLOv8...")

        df_modelo = self.df[self.df['modelo'] == 'yolov8'].copy()

        resultados = {
            'modelo': 'yolov8',
            'n_evaluaciones': len(df_modelo),
            'n_configuraciones': df_modelo['config_codigo'].nunique(),
            'factores': {}
        }

        for factor in ['tamano', 'config_sensibilidad']:
            col = f'factor_{factor}'
            if col not in df_modelo.columns:
                continue

            anova = self._calcular_anova(df_modelo, factor)

            stats_nivel = df_modelo.groupby(col)['iou'].agg([
                'mean', 'std', 'min', 'max', 'count'
            ]).reset_index()
            stats_nivel.columns = [factor, 'iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n']

            if factor == 'tamano':
                orden_tamano = ['nano', 'small', 'medium', 'large', 'xlarge']
                stats_nivel['orden'] = stats_nivel[factor].map(
                    {t: i for i, t in enumerate(orden_tamano)}
                )
                stats_nivel = stats_nivel.sort_values('orden').drop('orden', axis=1)

            posthoc = []
            if anova and anova.significativo:
                posthoc = self._calcular_posthoc_tukey(df_modelo, factor)

            resultados['factores'][factor] = {
                'anova': asdict(anova) if anova else None,
                'estadisticas_nivel': stats_nivel.to_dict('records'),
                'posthoc': [asdict(p) for p in posthoc]
            }

        if 'factor_tamano' in df_modelo.columns and 'factor_config_sensibilidad' in df_modelo.columns:
            interaccion = df_modelo.groupby(
                ['factor_tamano', 'factor_config_sensibilidad']
            )['iou'].agg(['mean', 'std', 'count']).reset_index()
            interaccion.columns = ['tamano', 'config_sensibilidad', 'iou_mean', 'iou_std', 'n']
            resultados['interaccion_tamano_sensibilidad'] = interaccion.to_dict('records')

        if 'factor_tamano' in df_modelo.columns:
            orden_tamano = ['nano', 'small', 'medium', 'large', 'xlarge']
            iou_por_tamano = df_modelo.groupby('factor_tamano')['iou'].mean()

            mejoras = []
            for i in range(1, len(orden_tamano)):
                if orden_tamano[i] in iou_por_tamano.index and orden_tamano[i-1] in iou_por_tamano.index:
                    mejora = iou_por_tamano[orden_tamano[i]] - iou_por_tamano[orden_tamano[i-1]]
                    mejoras.append({
                        'de': orden_tamano[i-1],
                        'a': orden_tamano[i],
                        'mejora_iou': float(mejora),
                        'mejora_porcentual': float(mejora / iou_por_tamano[orden_tamano[i-1]] * 100)
                    })

            resultados['analisis_rendimientos_decrecientes'] = mejoras

        mejor = df_modelo.groupby('config_codigo')['iou'].mean().idxmax()
        resultados['mejor_configuracion'] = mejor
        resultados['iou_mejor'] = float(df_modelo.groupby('config_codigo')['iou'].mean().max())

        return resultados

    # =========================================================================
    # COMPARACIÓN DE PARADIGMAS ARQUITECTÓNICOS
    # =========================================================================
    def comparar_paradigmas(self) -> Dict:
        """
        Compara los paradigmas arquitectónicos.

        Returns
        -------
        Dict
            Resultados de la comparación
        """
        logger.info("Comparando paradigmas arquitectónicos...")

        resultados = {
            'paradigmas': {},
            'comparaciones_pareadas': []
        }

        def asignar_paradigma(modelo):
            for paradigma, modelos in self.PARADIGMAS.items():
                if modelo in modelos:
                    return paradigma
            return 'unknown'

        self.df['paradigma'] = self.df['modelo'].apply(asignar_paradigma)

        for paradigma in self.PARADIGMAS.keys():
            df_paradigma = self.df[self.df['paradigma'] == paradigma]
            if len(df_paradigma) == 0:
                continue

            resultados['paradigmas'][paradigma] = {
                'modelos': self.PARADIGMAS[paradigma],
                'n_evaluaciones': len(df_paradigma),
                'n_configuraciones': df_paradigma['config_codigo'].nunique(),
                'iou_mean': float(df_paradigma['iou'].mean()),
                'iou_std': float(df_paradigma['iou'].std()),
                'iou_median': float(df_paradigma['iou'].median()),
                'iou_min': float(df_paradigma['iou'].min()),
                'iou_max': float(df_paradigma['iou'].max())
            }

        anova_paradigma = self._calcular_anova(self.df, 'paradigma')
        if anova_paradigma:
            resultados['anova_paradigmas'] = asdict(anova_paradigma)

        paradigmas_lista = list(self.PARADIGMAS.keys())
        for i in range(len(paradigmas_lista)):
            for j in range(i + 1, len(paradigmas_lista)):
                p1, p2 = paradigmas_lista[i], paradigmas_lista[j]

                datos_p1 = self.df[self.df['paradigma'] == p1]['iou']
                datos_p2 = self.df[self.df['paradigma'] == p2]['iou']

                if len(datos_p1) == 0 or len(datos_p2) == 0:
                    continue

                t_stat, p_val = stats.ttest_ind(datos_p1, datos_p2)

                pooled_std = np.sqrt((datos_p1.std()**2 + datos_p2.std()**2) / 2)
                cohens_d = (datos_p1.mean() - datos_p2.mean()) / pooled_std if pooled_std > 0 else 0

                resultados['comparaciones_pareadas'].append({
                    'paradigma_1': p1,
                    'paradigma_2': p2,
                    'media_1': float(datos_p1.mean()),
                    'media_2': float(datos_p2.mean()),
                    'diferencia': float(datos_p1.mean() - datos_p2.mean()),
                    't_statistic': float(t_stat),
                    'p_valor': float(p_val),
                    'cohens_d': float(cohens_d),
                    'significativo': p_val < 0.05
                })

        return resultados

    # =========================================================================
    # ANÁLISIS DE SENSIBILIDAD GLOBAL
    # =========================================================================
    def analizar_sensibilidad_umbrales(self) -> pd.DataFrame:
        """
        Analiza la sensibilidad a umbrales de confianza para todos los modelos.

        Returns
        -------
        pd.DataFrame
            Resultados de sensibilidad
        """
        logger.info("Analizando sensibilidad a umbrales...")

        resultados = []

        for modelo in ['bodypix', 'mask2former', 'oneformer']:
            df_modelo = self.df[self.df['modelo'] == modelo]

            col_umbral = 'factor_umbral' if modelo in ['oneformer', 'bodypix'] else 'factor_sensibilidad'

            if col_umbral not in df_modelo.columns:
                continue

            for umbral in df_modelo[col_umbral].dropna().unique():
                mask = df_modelo[col_umbral] == umbral
                datos = df_modelo.loc[mask, 'iou']

                resultados.append({
                    'modelo': modelo,
                    'umbral': umbral,
                    'iou_mean': float(datos.mean()),
                    'iou_std': float(datos.std()),
                    'iou_min': float(datos.min()),
                    'iou_max': float(datos.max()),
                    'n': len(datos)
                })

        return pd.DataFrame(resultados)

    def analizar_sensibilidad_tamano(self) -> pd.DataFrame:
        """
        Analiza la sensibilidad al tamaño del modelo.

        Returns
        -------
        pd.DataFrame
            Resultados de sensibilidad al tamaño
        """
        logger.info("Analizando sensibilidad al tamaño del modelo...")

        resultados = []

        # Incluir sam2 y sam2_prompts para análisis de tamaño
        for modelo_grupo in [['sam2', 'sam2_prompts'], ['yolov8']]:
            df_modelo = self.df[self.df['modelo'].isin(modelo_grupo)]
            col_tamano = 'factor_tamano'

            if col_tamano not in df_modelo.columns:
                continue

            # Nombre del modelo para el resultado
            nombre_modelo = 'sam2' if 'sam2' in modelo_grupo else modelo_grupo[0]

            for tamano in df_modelo[col_tamano].dropna().unique():
                mask = df_modelo[col_tamano] == tamano
                datos = df_modelo.loc[mask, 'iou']

                resultados.append({
                    'modelo': nombre_modelo,
                    'tamano': tamano,
                    'iou_mean': float(datos.mean()),
                    'iou_std': float(datos.std()),
                    'iou_min': float(datos.min()),
                    'iou_max': float(datos.max()),
                    'n': len(datos)
                })

        return pd.DataFrame(resultados)

    # =========================================================================
    # EJECUCIÓN COMPLETA
    # =========================================================================
    def ejecutar_analisis_completo(self) -> Dict:
        """
        Ejecuta todos los análisis de la Fase 2D.

        Returns
        -------
        Dict
            Todos los resultados
        """
        logger.info("="*60)
        logger.info("INICIANDO ANALISIS FASE 2D: CONFIGURACIONES")
        logger.info("="*60)

        # 1. Rankings
        logger.info("\n" + "-"*40)
        logger.info("1. Calculando rankings...")
        ranking_global = self.calcular_ranking_global(top_n=30)
        ranking_modelo = self.calcular_ranking_por_modelo()

        # 2. Análisis por modelo
        logger.info("\n" + "-"*40)
        logger.info("2. Analisis factorial por modelo...")
        self.resultados['bodypix'] = self.analizar_bodypix()
        self.resultados['mask2former'] = self.analizar_mask2former()
        self.resultados['oneformer'] = self.analizar_oneformer()
        self.resultados['sam2'] = self.analizar_sam2()
        self.resultados['yolov8'] = self.analizar_yolov8()

        # 3. Comparación de paradigmas
        logger.info("\n" + "-"*40)
        logger.info("3. Comparando paradigmas arquitectonicos...")
        self.resultados['paradigmas'] = self.comparar_paradigmas()

        # 4. Análisis de sensibilidad
        logger.info("\n" + "-"*40)
        logger.info("4. Analisis de sensibilidad...")
        self.resultados['sensibilidad_umbrales'] = self.analizar_sensibilidad_umbrales()
        self.resultados['sensibilidad_tamano'] = self.analizar_sensibilidad_tamano()

        # 5. Resumen de hallazgos
        logger.info("\n" + "-"*40)
        logger.info("5. Generando resumen...")
        self.resultados['resumen'] = self._generar_resumen()

        logger.info("\n" + "="*60)
        logger.info("ANALISIS FASE 2D COMPLETADO")
        logger.info("="*60)

        return self.resultados

    def _generar_resumen(self) -> Dict:
        """Genera un resumen de los hallazgos principales."""

        resumen = {
            'timestamp': datetime.now().isoformat(),
            'total_evaluaciones': len(self.df),
            'total_configuraciones': self.df['config_codigo'].nunique(),
            'total_fotografias': self.df['codigo_foto'].nunique(),
            'modelos_analizados': self.df['modelo'].unique().tolist()
        }

        mejor_por_modelo = self.resultados.get('ranking_por_modelo')
        if mejor_por_modelo is not None and len(mejor_por_modelo) > 0:
            mejor = mejor_por_modelo.iloc[0]
            resumen['mejor_modelo_global'] = {
                'modelo': mejor['modelo'],
                'configuracion': mejor['mejor_config'],
                'iou_mean': float(mejor['iou_mean'])
            }

        resumen['hallazgos_modelo'] = {}

        for modelo in ['bodypix', 'mask2former', 'oneformer', 'sam2', 'yolov8']:
            if modelo in self.resultados:
                res = self.resultados[modelo]
                hallazgo = {
                    'mejor_configuracion': res.get('mejor_configuracion'),
                    'iou_mejor': res.get('iou_mejor')
                }

                factores_sig = []
                for factor, datos in res.get('factores', {}).items():
                    if datos.get('anova') and datos['anova'].get('significativo'):
                        factores_sig.append({
                            'factor': factor,
                            'eta_squared': datos['anova']['eta_squared'],
                            'interpretacion': datos['anova']['interpretacion_efecto']
                        })
                hallazgo['factores_significativos'] = factores_sig

                resumen['hallazgos_modelo'][modelo] = hallazgo

        return resumen

    # =========================================================================
    # EXPORTACIÓN DE RESULTADOS
    # =========================================================================
    @staticmethod
    def _convertir_tipos_json(obj):
        """Convierte tipos numpy a tipos nativos de Python para JSON."""
        if isinstance(obj, dict):
            return {k: AnalizadorConfiguraciones._convertir_tipos_json(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [AnalizadorConfiguraciones._convertir_tipos_json(item) for item in obj]
        elif isinstance(obj, (np.integer, np.int64, np.int32)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64, np.float32)):
            return float(obj)
        elif isinstance(obj, (np.bool_, bool)):
            return bool(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif pd.isna(obj):
            return None
        return obj

    def exportar_resultados(self):
        """Exporta todos los resultados a archivos."""

        logger.info("Exportando resultados...")
        self.rutas.crear_directorios()

        # 1. Ranking global
        if 'ranking_global' in self.resultados:
            ruta = os.path.join(self.rutas.salida, 'ranking_global_top30.csv')
            self.resultados['ranking_global'].head(30).to_csv(ruta, index=False)
            logger.info(f"  -> {ruta}")

        # 2. Ranking por modelo
        if 'ranking_por_modelo' in self.resultados:
            ruta = os.path.join(self.rutas.salida, 'ranking_por_modelo.csv')
            self.resultados['ranking_por_modelo'].to_csv(ruta, index=False)
            logger.info(f"  -> {ruta}")

        # 3. Análisis por modelo (JSON)
        for modelo in ['bodypix', 'mask2former', 'oneformer', 'sam2', 'yolov8']:
            if modelo in self.resultados:
                ruta = os.path.join(self.rutas.salida, f'factores_{modelo}.json')
                datos_convertidos = self._convertir_tipos_json(self.resultados[modelo])
                with open(ruta, 'w', encoding='utf-8') as f:
                    json.dump(datos_convertidos, f, indent=2, ensure_ascii=False)
                logger.info(f"  -> {ruta}")

        # 4. Comparación de paradigmas
        if 'paradigmas' in self.resultados:
            ruta = os.path.join(self.rutas.salida, 'comparacion_paradigmas.json')
            datos_convertidos = self._convertir_tipos_json(self.resultados['paradigmas'])
            with open(ruta, 'w', encoding='utf-8') as f:
                json.dump(datos_convertidos, f, indent=2, ensure_ascii=False)
            logger.info(f"  -> {ruta}")

        # 5. Sensibilidad umbrales
        if 'sensibilidad_umbrales' in self.resultados:
            df_sens = self.resultados['sensibilidad_umbrales']
            if isinstance(df_sens, pd.DataFrame) and len(df_sens) > 0:
                ruta = os.path.join(self.rutas.salida, 'sensibilidad_umbrales.csv')
                df_sens.to_csv(ruta, index=False)
                logger.info(f"  -> {ruta}")

        # 6. Sensibilidad tamaño
        if 'sensibilidad_tamano' in self.resultados:
            df_sens = self.resultados['sensibilidad_tamano']
            if isinstance(df_sens, pd.DataFrame) and len(df_sens) > 0:
                ruta = os.path.join(self.rutas.salida, 'sensibilidad_tamano.csv')
                df_sens.to_csv(ruta, index=False)
                logger.info(f"  -> {ruta}")

        # 7. Resumen general
        if 'resumen' in self.resultados:
            ruta = os.path.join(self.rutas.salida, 'resumen_fase2d.json')
            datos_convertidos = self._convertir_tipos_json(self.resultados['resumen'])
            with open(ruta, 'w', encoding='utf-8') as f:
                json.dump(datos_convertidos, f, indent=2, ensure_ascii=False)
            logger.info(f"  -> {ruta}")

        # 8. ANOVA consolidado
        anova_consolidado = []
        for modelo in ['bodypix', 'mask2former', 'oneformer', 'sam2', 'yolov8']:
            if modelo in self.resultados:
                for factor, datos in self.resultados[modelo].get('factores', {}).items():
                    if datos.get('anova'):
                        anova_data = datos['anova'].copy()
                        anova_data['modelo'] = modelo
                        anova_consolidado.append(anova_data)

        if anova_consolidado:
            df_anova = pd.DataFrame(anova_consolidado)
            ruta = os.path.join(self.rutas.salida, 'anova_por_modelo.csv')
            df_anova.to_csv(ruta, index=False)
            logger.info(f"  -> {ruta}")

        # 9. Post-hoc Tukey consolidado
        posthoc_consolidado = []
        for modelo in ['bodypix', 'mask2former', 'oneformer', 'sam2', 'yolov8']:
            if modelo in self.resultados:
                for factor, datos in self.resultados[modelo].get('factores', {}).items():
                    for ph in datos.get('posthoc', []):
                        ph_data = ph.copy()
                        ph_data['modelo'] = modelo
                        ph_data['factor'] = factor
                        posthoc_consolidado.append(ph_data)

        if posthoc_consolidado:
            df_posthoc = pd.DataFrame(posthoc_consolidado)
            ruta = os.path.join(self.rutas.salida, 'posthoc_tukey.csv')
            df_posthoc.to_csv(ruta, index=False)
            logger.info(f"  -> {ruta}")

        logger.info("Exportacion completada.")

In [7]:
# =============================================================================
# FUNCIÓN PRINCIPAL
# =============================================================================
def ejecutar_fase2d(ruta_metricas: str = None) -> Dict:
    """
    Ejecuta la Fase 2D completa.

    Parameters
    ----------
    ruta_metricas : str, optional
        Ruta al archivo metricas_fusionadas.csv

    Returns
    -------
    Dict
        Resultados del análisis
    """
    # Configurar rutas
    rutas = ConfiguracionRutas()

    # Determinar ruta de entrada
    if ruta_metricas is None:
        ruta_metricas = os.path.join(rutas.datos_entrada, 'metricas_fusionadas.csv')

    logger.info(f"Cargando datos desde: {ruta_metricas}")

    # Cargar datos
    if not os.path.exists(ruta_metricas):
        raise FileNotFoundError(f"No se encontró el archivo: {ruta_metricas}")

    df = pd.read_csv(ruta_metricas)
    logger.info(f"Datos cargados: {len(df)} filas, {len(df.columns)} columnas")

    # Crear analizador y ejecutar
    analizador = AnalizadorConfiguraciones(df, rutas)
    resultados = analizador.ejecutar_analisis_completo()

    # Exportar resultados
    analizador.exportar_resultados()

    return resultados

In [8]:
# =============================================================================
# MAIN
# =============================================================================
if __name__ == '__main__':
    from google.colab import drive
    drive.mount('/content/drive')

    # Ejecutar análisis
    resultados = ejecutar_fase2d()

    # Mostrar resumen
    print("\n" + "="*60)
    print("RESUMEN DE FASE 2D")
    print("="*60)

    if 'resumen' in resultados:
        resumen = resultados['resumen']
        print(f"Total evaluaciones: {resumen['total_evaluaciones']}")
        print(f"Total configuraciones: {resumen['total_configuraciones']}")
        print(f"Modelos analizados: {resumen['modelos_analizados']}")

        if 'mejor_modelo_global' in resumen:
            mejor = resumen['mejor_modelo_global']
            print(f"\nMejor modelo: {mejor['modelo']}")
            print(f"Mejor configuración: {mejor['configuracion']}")
            print(f"IoU: {mejor['iou_mean']:.4f}")

[22:15:09] INFO - Cargando datos desde: /content/drive/MyDrive/TFM/3_Analisis/fase2b_correlaciones/metricas_fusionadas.csv


Mounted at /content/drive


[22:15:11] INFO - Datos cargados: 2360 filas, 220 columnas
[22:15:11] INFO - Extrayendo factores de configuraciones...
[22:15:11] INFO -   yolov8.factor_tamano: ['large', 'medium', 'nano', 'small', 'xlarge']
[22:15:11] INFO -   yolov8.factor_config_sensibilidad: ['balanced', 'fast', 'quality', 'sensitive']
[22:15:11] INFO -   bodypix.factor_multiplicador: ['050', '075']
[22:15:11] INFO -   bodypix.factor_nivel_sensibilidad: ['baja_sensibilidad', 'sensibilidad_alta', 'sensibilidad_media', 'ultra_sensible']
[22:15:11] INFO -   bodypix.factor_umbral: ['1', '15', '2', '25', '3', '4', '5']
[22:15:11] INFO -   mask2former.factor_backbone: ['base', 'large', 'tiny']
[22:15:11] INFO -   mask2former.factor_dataset: ['ade', 'coco']
[22:15:11] INFO -   mask2former.factor_sensibilidad: ['alta_sensibilidad', 'baja_sensibilidad', 'experimental_coco', 'maxima_sensibilidad', 'media_sensibilidad']
[22:15:11] INFO -   oneformer.factor_umbral: ['t040', 't060', 't075', 't085']
[22:15:11] INFO -   oneformer


RESUMEN DE FASE 2D
Total evaluaciones: 2360
Total configuraciones: 143
Modelos analizados: ['yolov8', 'bodypix', 'mask2former', 'oneformer', 'sam2', 'sam2_prompts']

Mejor modelo: oneformer
Mejor configuración: oneformer_coco_swin_semantic_t040
IoU: 0.9674
